# Final Clean Test: Multi-Turn Commitment

**Hypothesis**: Linkage strength controls persistence/commitment across turns.

**Settings**: 
- `continuity_gain=0.3` (minimal interference)
- Progress bars for visibility
- Token streaming for real-time feedback

**Primary Test**: Stance revision under contradiction

In [ ]:
import torch
import sys
import os
from tqdm.auto import tqdm
from transformers import TextStreamer

sys.path.append(os.path.abspath('../'))
from src.models.model_injection import create_triplet_model

print("Loading model (this takes ~2 minutes)...")
model, tokenizer, controller = create_triplet_model()
print("✓ Model loaded successfully")

In [ ]:
def run_dialogue(prompts, linkage_mode, max_tokens=80):
    """Runs a multi-turn dialogue with progress tracking."""
    history = ""
    responses = []
    
    print(f"\n{'='*60}")
    print(f"MODE: {linkage_mode.upper()}")
    print(f"{'='*60}\n")
    
    streamer = TextStreamer(tokenizer, skip_prompt=True)
    
    # Progress bar for turns
    for i in tqdm(range(len(prompts)), desc=f"{linkage_mode} mode", ncols=80):
        prompt = prompts[i]
        input_text = f"{history}User: {prompt}\nAssistant:"
        
        print(f"\n[Turn {i+1}/{len(prompts)}] User: {prompt}")
        print("Assistant: ", end="", flush=True)
        
        response, _ = controller.generate_with_linkage(
            input_text,
            max_tokens=max_tokens,
            linkage_mode=linkage_mode,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            streamer=streamer
        )
        
        history = f"{input_text} {response}\n"
        responses.append((prompt, response))
    
    print(f"\n{'='*60}")
    print(f"Completed {len(prompts)} turns")
    print(f"{'='*60}\n")
    
    return responses

## Primary Test: Commitment Under Contradiction

We ask the model to take a stance, then challenge it. Does linkage affect how easily it revises?

In [ ]:
commitment_test = [
    "Take a firm stance: Is remote work better than office work? One sentence.",
    "List the strongest arguments against your stance.",
    "Revise your stance. One sentence.",
    "Point out any contradiction between your first and revised stance.",
    "Summarize your final position in one sentence."
]

print("\n🔗 Running LINKED mode (continuity_gain=0.3)...")
linked_responses = run_dialogue(commitment_test, linkage_mode="full", max_tokens=80)

print("\n👁️ Running OBSERVER mode (no continuity)...")
observer_responses = run_dialogue(commitment_test, linkage_mode="observer", max_tokens=80)

## Analysis: Side-by-Side Comparison

In [ ]:
from IPython.display import display, HTML
import pandas as pd

comparison_data = []
for i in range(len(commitment_test)):
    comparison_data.append({
        'Turn': i+1,
        'Question': commitment_test[i],
        'Linked': linked_responses[i][1],
        'Observer': observer_responses[i][1]
    })

df = pd.DataFrame(comparison_data)
display(df)

## Quick Metrics

Look for:
- **Turn 3 (Revision)**: Does linked mode resist changing stance?
- **Turn 4 (Contradiction)**: Does linked mode acknowledge or deny contradiction?
- **Response length**: Is one mode more verbose/defensive?

In [ ]:
print("Response Length Comparison:")
print(f"Linked avg tokens: {sum(len(r[1].split()) for r in linked_responses) / len(linked_responses):.1f}")
print(f"Observer avg tokens: {sum(len(r[1].split()) for r in observer_responses) / len(observer_responses):.1f}")

print("\nStance Revision (Turn 3):")
print(f"Linked: {linked_responses[2][1][:100]}...")
print(f"Observer: {observer_responses[2][1][:100]}...")